In [1]:
import pandas as pd
import os

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

from fuzzywuzzy import process

from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch

import io
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

In [2]:
# Cargar el archivo Excel
file_path = '/Users/floroppen/Desktop/Python/Scripts/Vendemos.xlsx'
df = pd.read_excel(file_path, header=1)  # Las columnas están en la fila 2, que es la fila 1 en índice 0-based

# Mostrar las primeras filas para verificar la estructura y las columnas
df.head()

,Producto,Entrega,Precio Venta (USD),% de descuento aplicado al precio nuevo o similar,precio nuevo o similar (dolar) c/u,Disponibilidad,Cantidad,Observaciones,Link nueva / equivalente (no son mias estas publicaciones),Fotos reales
0,Asus zenpad Z10,Inmediata,59.5,0.30,85.0,Vendido,1,puede agregarse un chip para 4G. tiene 32gb de...,https://www.ebay.com/itm/225812413452?epid=238...,https://drive.google.com/drive/folders/1DnPYwl...
1,Auriculares Razer Kraken Kitty,Inmediata,168.0,0.30,240.0,Disponible,1,Auriculares Razer Kraken Kitty con luz rgb LED...,https://www.mercadolibre.com.ar/auriculares-ga...,https://drive.google.com/drive/folders/1reT_o2...
2,Batidora Mondial,Inmediata,30.0,0.30,44.0,Disponible,1,"Batidora Mondial Bella Massa Inox, 400w de pot...",https://www.mercadolibre.com.ar/batidora-de-pi...,https://drive.google.com/drive/folders/1iyIVWn...
3,Bicicleta fija,Inmediata,130.0,0.60,333.0,Disponible,1,"Tiene algunas marcas por maravillosos felinos,...",https://articulo.mercadolibre.com.ar/MLA-60235...,https://drive.google.com/drive/folders/1wHpQb4...
4,Bleff,Inmediata,14.1,0.25,18.8,Vendido,1,"Juego de Mesa Bleff, cerrado",https://www.mercadolibre.com.ar/juego-de-mesa-...,https://drive.google.com/drive/folders/1wLJDZh...


In [3]:
df = df[['Producto', 'Entrega','Disponibilidad', 'Precio Venta (USD)', 'Observaciones', 'Fotos reales']]
df.head(14)

,Producto,Entrega,Disponibilidad,Precio Venta (USD),Observaciones,Fotos reales
0,Asus zenpad Z10,Inmediata,Vendido,59.5,puede agregarse un chip para 4G. tiene 32gb de...,https://drive.google.com/drive/folders/1DnPYwl...
1,Auriculares Razer Kraken Kitty,Inmediata,Disponible,168.0,Auriculares Razer Kraken Kitty con luz rgb LED...,https://drive.google.com/drive/folders/1reT_o2...
2,Batidora Mondial,Inmediata,Disponible,30.0,"Batidora Mondial Bella Massa Inox, 400w de pot...",https://drive.google.com/drive/folders/1iyIVWn...
3,Bicicleta fija,Inmediata,Disponible,130.0,"Tiene algunas marcas por maravillosos felinos,...",https://drive.google.com/drive/folders/1wHpQb4...
4,Bleff,Inmediata,Vendido,14.1,"Juego de Mesa Bleff, cerrado",https://drive.google.com/drive/folders/1wLJDZh...
5,Botines Adidas Golleto,Inmediata,Disponible,21.0,"Botin Adidas Goletto VI Negro/Blanco talle 37,...",https://drive.google.com/drive/folders/1h5DzTX...
6,Brisingr y El Legado,Inmediata,Disponible,15.0,"Colección el legado III y IV, tapa blanda",https://drive.google.com/drive/folders/1-DArOB...
7,Cafetera sage bambino plus,Junio,Disponible,385.0,Cafetera espresso semiautomática,https://drive.google.com/drive/folders/103ksua...
8,Calentador de agua,Inmediata,Disponible,60.0,"Calentador de agua sumergible, para 5 galones,...",https://drive.google.com/drive/folders/1KLC2sP...
9,Camara profesional Canon 80D,Inmediata,Disponible,900.0,La compré usada y no la usé casi nada al final...,https://drive.google.com/drive/folders/18_zCfz...


In [4]:
df.shape

(63, 6)

In [5]:
df2 = df[(df['Entrega'] == 'Inmediata') & (df['Disponibilidad'] == 'Disponible')].copy()
df2['Folder ID'] = df2['Fotos reales'].apply(lambda x: x.split('/')[-1])
df2.head(20)

,Producto,Entrega,Disponibilidad,Precio Venta (USD),Observaciones,Fotos reales,Folder ID
1,Auriculares Razer Kraken Kitty,Inmediata,Disponible,168.0,Auriculares Razer Kraken Kitty con luz rgb LED...,https://drive.google.com/drive/folders/1reT_o2...,1reT_o2GguU4Rwr4ZJ5IiEubJz26OrtHz?usp=sharing
2,Batidora Mondial,Inmediata,Disponible,30.0,"Batidora Mondial Bella Massa Inox, 400w de pot...",https://drive.google.com/drive/folders/1iyIVWn...,1iyIVWn9xKFSsKe8rQ-F1Ep7KSsB-JY2t
3,Bicicleta fija,Inmediata,Disponible,130.0,"Tiene algunas marcas por maravillosos felinos,...",https://drive.google.com/drive/folders/1wHpQb4...,1wHpQb43UDGrFp51r_4ph6mU879M-rGbo
5,Botines Adidas Golleto,Inmediata,Disponible,21.0,"Botin Adidas Goletto VI Negro/Blanco talle 37,...",https://drive.google.com/drive/folders/1h5DzTX...,1h5DzTXsPwGNbgF5lsNHls03AxEFDJ7Ip
6,Brisingr y El Legado,Inmediata,Disponible,15.0,"Colección el legado III y IV, tapa blanda",https://drive.google.com/drive/folders/1-DArOB...,1-DArOBPOYe9HbgkPIqX_CBr__SFic6K_
8,Calentador de agua,Inmediata,Disponible,60.0,"Calentador de agua sumergible, para 5 galones,...",https://drive.google.com/drive/folders/1KLC2sP...,1KLC2sPBFvuMr6lzRoNn8ja4C1-PmvuOt
9,Camara profesional Canon 80D,Inmediata,Disponible,900.0,La compré usada y no la usé casi nada al final...,https://drive.google.com/drive/folders/18_zCfz...,18_zCfz5gQADgjH3Qsz9qsJM-H4rC8XEf
10,Lente camara canon 10-18mm,Inmediata,Disponible,165.0,"Canon EF-S 10-18mm f/4.5-5.6 is STM Lens, prim...",https://drive.google.com/drive/folders/16o64rv...,16o64rvttQ4SiZ36Mu8_huwBcZ9nocVmt
11,Lente camara canon EOS 24mm,Inmediata,Disponible,150.0,"Canon EF-S 24mm f/2.8 STM Lens, primera mano",https://drive.google.com/drive/folders/1D-i9E_...,1D-i9E_faI34cndwNCyjiCqpjL-3wnbyv
12,Cargador portatil APC M10,Inmediata,Disponible,36.0,Cargador portatil APC M10,https://drive.google.com/drive/folders/1-8q1VV...,1-8q1VVyEv8TUxalRUO9wo7fR8oH4KGHY?usp=drive_link


In [6]:
df2.shape

(29, 7)

In [7]:
# Inicializa GoogleAuth
gauth = GoogleAuth()

# Carga el archivo client_secrets.json para autenticarse con Google
gauth.LoadClientConfigFile("client_secrets.json")

# Realiza la autenticación (esto abrirá una ventana en tu navegador para que otorgues acceso a tu aplicación)
gauth.LocalWebserverAuth()

# Crea un objeto GoogleDrive utilizando la autenticación realizada
drive = GoogleDrive(gauth)

#Credenciales
creds = Credentials.from_authorized_user_file('token.json')
service = build('drive', 'v3', credentials=creds)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=884899627283-7i94a51dbfsb8i77ak27qoenqukqekeg.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [8]:
#Como el nombre de las cosas en df2 y las carpetas en Drive no son identicas, primero voy a listar el nombre de las carpetas y sus ids

def obtener_nombres_ids_carpetas(folder_id):
    """Obtiene los nombres e IDs de las carpetas dentro del folder_id proporcionado."""
    carpetas = []
    query = f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false"
    file_list = drive.ListFile({'q': query}).GetList()
    
    for file in file_list:
        carpetas.append((file['title'], file['id']))  # Cambia 'title' por 'name' si es necesario
    
    return carpetas

# Reemplaza 'tu_folder_id_aquí' con el ID real de tu carpeta principal
folder_id_principal = '1FZNsil8joasf0Yp7ccthhDWTAl8pkcnr'
carpetas = obtener_nombres_ids_carpetas(folder_id_principal)

print(carpetas)

[('ROLLERS', '1D16o3zeYynoEOi0gdWRzwZ58LlXaqU_u'), ('TECLADO_Y_MOUSE_ROSA_ MAGEGGEE_GK710', '17Q0_FwqpVvPN-xaH0Ud0wdlBItvG8mLr'), ('3DS', '13IXfP-JSSjwpOhr51bNaxdk4CaJhwiKJ'), ('RAZER_KRAKEN_KITTY', '1reT_o2GguU4Rwr4ZJ5IiEubJz26OrtHz'), ('CUADRO_DE_JUEGOS_DE_FROMSOFTWARE', '1LoBPvhsRRKtJJqP6DEbnpICEvucZVLAi'), ('ASUS_ZENPAD_10', '1DnPYwlKNuTd4GrQ4rnaukwvJgSV67k3I'), ('TECLADO_PIANO_YAMAHA', '1KjLWCXpnyPalreuGPcf9ttCcCO559DK2'), ('CUADRO_CANVAS__DE_MAJORAS_MASK', '1zmKzNQRrFEhTGTWXL75aWfQNUx6232iK'), ('BICI_FIJA_', '1wHpQb43UDGrFp51r_4ph6mU879M-rGbo'), ('WII', '1AoCTRRjrY16X5af5IzQ1d5Y_BcR96JHf'), ('RASPBERRY_PI_4_B_4GB_RAM', '1K6_Rf4oTiMn12m782rY5dR9t2COmp8Od'), ('CARGADOR_PORTATIL_APC_M10', '1-8q1VVyEv8TUxalRUO9wo7fR8oH4KGHY'), ('LENTE_CAMARA_10-18MM', '16o64rvttQ4SiZ36Mu8_huwBcZ9nocVmt'), ('SILLON_GRIS_3_CUERPOS_ANTIDESGARRO_CAMASTRO', '1Hh_gHyeQd2DEW_013nk93hZqjepWk8mV'), ('MESA_EXTERIOR', '1fWbu_JFdjUOha0r7LZ274d4pNSTN_xTE'), ('RING_FIT', '114_9quRH54Yp4-RVsz3w_HoIxe12C2_v'), ('DEK

In [9]:
#Voy a asociar los productos con las carpetas e ids correspondientes

def encontrar_carpeta_similar(nombre_producto, carpetas):
    # Extrae solo los nombres de las carpetas para la comparación
    nombres_carpetas = [carpeta[0] for carpeta in carpetas]
    
    # Encuentra el nombre de la carpeta más similar
    nombre_carpeta_similar, _ = process.extractOne(nombre_producto, nombres_carpetas)
    
    # Encuentra la tupla completa (nombre e ID) para la carpeta coincidente
    carpeta_coincidente = next((carpeta for carpeta in carpetas if carpeta[0] == nombre_carpeta_similar), None)
    
    return carpeta_coincidente

# Ahora, itera sobre los nombres de los productos en df2 y encuentra las carpetas correspondientes
resultados = []
for nombre_producto in df2['Producto']:
    carpeta_similar = encontrar_carpeta_similar(nombre_producto, carpetas)
    if carpeta_similar:
        # Guarda el resultado como una tupla de (nombre_producto, nombre_carpeta, id_carpeta)
        resultados.append((nombre_producto, carpeta_similar[0], carpeta_similar[1]))

# resultados contiene una lista de productos junto con el nombre e ID de la carpeta más similar en Drive
print(resultados)

[('Auriculares Razer Kraken Kitty', 'RAZER_KRAKEN_KITTY', '1reT_o2GguU4Rwr4ZJ5IiEubJz26OrtHz'), ('Batidora Mondial', 'BATIDORA_MONDIAL', '1iyIVWn9xKFSsKe8rQ-F1Ep7KSsB-JY2t'), ('Bicicleta fija', 'BICI_FIJA_', '1wHpQb43UDGrFp51r_4ph6mU879M-rGbo'), ('Botines Adidas Golleto', 'BOTINES_ADIDAS_GOLLETO', '1h5DzTXsPwGNbgF5lsNHls03AxEFDJ7Ip'), ('Brisingr y El Legado', 'BRISINGR_Y_EL_LEGADO', '1-DArOBPOYe9HbgkPIqX_CBr__SFic6K_'), ('Calentador de agua ', 'CALENTADOR_DE_AGUA_', '1KLC2sPBFvuMr6lzRoNn8ja4C1-PmvuOt'), ('Camara profesional Canon 80D', 'CAMARA_PROFESIONAL_CANON_80D', '18_zCfz5gQADgjH3Qsz9qsJM-H4rC8XEf'), ('Lente camara canon 10-18mm', 'LENTE_CAMARA_10-18MM', '16o64rvttQ4SiZ36Mu8_huwBcZ9nocVmt'), ('Lente camara canon EOS 24mm', 'LENTE_CAMARA_CANON_EOS_24MM', '1D-i9E_faI34cndwNCyjiCqpjL-3wnbyv'), ('Cargador portatil APC M10', 'CARGADOR_PORTATIL_APC_M10', '1-8q1VVyEv8TUxalRUO9wo7fR8oH4KGHY'), ('Casio fx-95ES PLUS', 'CASIO_FX-95ES_PLUS', '1hnjdXml6iwAKXpOGDT-_McSthXzqiC8N'), ('Casio fx-991

In [10]:
#Ahora voy a descargar las imagenes localmente
def descargar_imagenes(service, resultados):
    os.makedirs('imagenes_descargadas', exist_ok=True)
    
    for producto, _, carpeta_id in resultados:
        query = f"'{carpeta_id}' in parents and mimeType contains 'image/' and trashed=false"
        response = service.files().list(q=query, fields="files(id, name)").execute()
        items = response.get('files', [])
        
        if not items:
            print(f"No se encontraron imágenes en la carpeta: {producto}")
        else:
            for i, item in enumerate(items):
                file_id = item['id']
                original_file_name = item['name']
                file_extension = os.path.splitext(original_file_name)[1]  # Extrae la extensión del archivo original
                normalized_product_name = producto.replace(' ', '_').replace('/', '_')
                
                # Construye el nuevo nombre del archivo manteniendo la extensión original
                new_file_name = f"{normalized_product_name}_{i+1}{file_extension}"
                file_path = os.path.join('imagenes_descargadas', new_file_name)
                
                if os.path.exists(file_path):
                    print(f"El archivo ya existe: {file_path}")
                    continue
                
                request = service.files().get_media(fileId=file_id)
                fh = io.FileIO(file_path, 'wb')
                downloader = MediaIoBaseDownload(fh, request)
                
                done = False
                while not done:
                    _, done = downloader.next_chunk()
                
                print(f"Descargada '{file_path}'")

descargar_imagenes(service, resultados)

Descargada 'imagenes_descargadas/Auriculares_Razer_Kraken_Kitty_1.HEIC'
Descargada 'imagenes_descargadas/Batidora_Mondial_1.HEIC'
Descargada 'imagenes_descargadas/Bicicleta_fija_1.HEIC'
Descargada 'imagenes_descargadas/Bicicleta_fija_2.HEIC'
Descargada 'imagenes_descargadas/Bicicleta_fija_3.HEIC'
Descargada 'imagenes_descargadas/Botines_Adidas_Golleto_1.HEIC'
Descargada 'imagenes_descargadas/Botines_Adidas_Golleto_2.HEIC'
Descargada 'imagenes_descargadas/Botines_Adidas_Golleto_3.HEIC'
Descargada 'imagenes_descargadas/Botines_Adidas_Golleto_4.HEIC'
Descargada 'imagenes_descargadas/Brisingr_y_El_Legado_1.HEIC'
Descargada 'imagenes_descargadas/Brisingr_y_El_Legado_2.HEIC'
Descargada 'imagenes_descargadas/Brisingr_y_El_Legado_3.HEIC'
Descargada 'imagenes_descargadas/Calentador_de_agua__1.HEIC'
Descargada 'imagenes_descargadas/Camara_profesional_Canon_80D_1.HEIC'
Descargada 'imagenes_descargadas/Lente_camara_canon_10-18mm_1.HEIC'
Descargada 'imagenes_descargadas/Lente_camara_canon_EOS_24mm_

In [11]:
from PIL import Image as PILImage
from pillow_heif import register_heif_opener
register_heif_opener()

directorio_imagenes = 'imagenes_descargadas' 
directorio_salida = 'imagenes_convertidas'

# Crea el directorio de salida si no existe
os.makedirs(directorio_salida, exist_ok=True)

# Lista todos los archivos en el directorio de imágenes
for archivo in os.listdir(directorio_imagenes):
    # Verifica si el archivo es una imagen HEIC
    if archivo.endswith(".HEIC") or archivo.endswith(".heic"):
        ruta_completa_heic = os.path.join(directorio_imagenes, archivo)
        nombre_base = os.path.splitext(archivo)[0]
        ruta_completa_jpg = os.path.join(directorio_salida, nombre_base + ".jpg")
        
        # Abre y convierte la imagen
        with PILImage.open(ruta_completa_heic) as img:
            img.convert("RGB").save(ruta_completa_jpg, "JPEG")
            print(f"Convertido {archivo} a JPG.")

Convertido Reposeras_Just_Home_Collection_Sao_Paulo.__3.HEIC a JPG.
Convertido Lente_camara_canon_10-18mm_1.HEIC a JPG.
Convertido Venom_chiquito_2.HEIC a JPG.
Convertido Bicicleta_fija_2.HEIC a JPG.
Convertido Libros_LOTR_1.HEIC a JPG.
Convertido Mesa_escritorio_eléctrica_regulable_hafele_2_motores_125kg_1.heic a JPG.
Convertido Casio_fx-95ES_PLUS_1.HEIC a JPG.
Convertido Bicicleta_fija_3.HEIC a JPG.
Convertido Camara_profesional_Canon_80D_1.HEIC a JPG.
Convertido Reposeras_Just_Home_Collection_Sao_Paulo.__2.HEIC a JPG.
Convertido Soporte_Gadnic_monitor_1.HEIC a JPG.
Convertido Lente_camara_canon_EOS_24mm_1.HEIC a JPG.
Convertido Ring_Fit_1.HEIC a JPG.
Convertido Brisingr_y_El_Legado_1.HEIC a JPG.
Convertido Batidora_Mondial_1.HEIC a JPG.
Convertido PS_Vita_hackeada__1.HEIC a JPG.
Convertido TEG_1.HEIC a JPG.
Convertido Nintendo_3DS_1.HEIC a JPG.
Convertido Monitor_Samsung_S34J550WQL_led_34.1"_dark_blue_gray_100V_240V_1.HEIC a JPG.
Convertido Deku_1.HEIC a JPG.
Convertido Cargador_por

In [15]:
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, KeepTogether
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from PIL import Image as PILImage
import os
from reportlab.lib.enums import TA_JUSTIFY

documento = SimpleDocTemplate("Venta_Productos.pdf", pagesize=letter)
flowables = []

styles = getSampleStyleSheet()
style_normal = styles['Normal']
style_normal.fontSize = 12
style_normal.alignment = TA_JUSTIFY  # Ajusta el texto para que esté justificado

# Añade el texto introductorio
texto_introductorio = """
Somos Maxi y Flor. Nos vamos del país y estamos vendiendo algunas cosas. 
El precio que figura está en dólares, pero se puede pagar en pesos al valor venta en dolarhoy.com. 
Se acepta transferencia/efectivo. Whatsapp de contacto: 11-3497-6502.
En el mes de junio se agregarán más productos a este listado.
Se retira por Nordelta.
"""
parrafo_introductorio = Paragraph(texto_introductorio, style_normal)
flowables.append(parrafo_introductorio)
flowables.append(Spacer(1, 0.5 * inch))

for index, fila in df2.iterrows():
    elementos_producto = []  # Esta lista almacenará temporalmente los elementos de cada producto
    
    # Información del producto
    texto_producto = f"<b>{fila['Producto']}</b> - ${fila['Precio Venta (USD)']}<br/>{fila['Observaciones']}"
    parrafo_producto = Paragraph(texto_producto, style_normal)
    elementos_producto.append(parrafo_producto)
    elementos_producto.append(Spacer(1, 0.2 * inch))
    
    # Lista para almacenar las imágenes del producto
    imagenes_producto = []
    
    nombre_producto_normalizado = fila["Producto"].replace(' ', '_')
    for n in range(1, 5):  # Ajusta según el número de imágenes por producto
        nombre_imagen = f'imagenes_convertidas/{nombre_producto_normalizado}_{n}.jpg'
        if not os.path.exists(nombre_imagen):
            continue
        try:
            with PILImage.open(nombre_imagen) as pil_img:
                ancho, alto = pil_img.size
                if ancho > alto:
                    img = Image(nombre_imagen, width=2*inch, height=1.5*inch)
                else:
                    img = Image(nombre_imagen, width=1.5*inch, height=2*inch)
            imagenes_producto.append(img)
        except Exception as e:
            pass
    
    # Si hay imágenes, crea una tabla para ellas y agrégala a los elementos del producto
    if imagenes_producto:
        tabla_imagenes = Table([imagenes_producto], colWidths=[2*inch for img in imagenes_producto], hAlign='CENTER')
        elementos_producto.append(tabla_imagenes)
        elementos_producto.append(Spacer(1, 0.2 * inch))
    
    # Usa KeepTogether en los elementos del producto
    producto_completo = KeepTogether(elementos_producto)
    flowables.append(producto_completo)

documento.build(flowables)